##Face Mask Detection by Denys Sikorskyi

**Downloading data**

Firstly, we create two folder on Google Drive: train and validation. Both of them are also divided in two folders: with and without. Performing rescaling, rotation, zoom, flip and shift range for training folder. Performing only rescaling for validation.

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
train_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Face Mask Detection/train'
val_path = '/gdrive/My Drive/Colab Notebooks/Datafiles/Face Mask Detection/validation'
IMG_HEIGHT, IMG_WIDTH = 224, 224
batch_size = 8
epochs = 20
train_image_generator = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                ) 
validation_image_generator = ImageDataGenerator(rescale=1./255)
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_path,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=val_path,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')

Mounted at /gdrive
/gdrive
Found 5609 images belonging to 2 classes.
Found 1944 images belonging to 2 classes.


**Creating model**

Importing InceptionV3 model for implemetentantion. In first session, we freeze inception layers and train top layers. Then, we freeze top layers and train inception layers. 

In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.0000001)
callbacks = [earlystop,learning_rate_reduction]
# train the model on the new data for a few epochs
history = model.fit(train_data_gen, 
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=1944//batch_size,
    steps_per_epoch=5609//batch_size,
    callbacks=callbacks)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
model.summary()

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit(train_data_gen, 
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=1944//batch_size,
    steps_per_epoch=5609//batch_size,
    callbacks=callbacks)
model.save("/content/Face_Mask_Detection.h5")

87916544/87910968 [==============================] - 0s 0us/step
Epoch 1/20
701/701 [==============================] - ETA: 0s - loss: 0.4005 - accuracy: 0.9368

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


701/701 [==============================] - 1990s 3s/step - loss: 0.4001 - accuracy: 0.9368 - val_loss: 0.0645 - val_accuracy: 0.9810
Epoch 2/20
701/701 [==============================] - 90s 129ms/step - loss: 0.1133 - accuracy: 0.9847 - val_loss: 0.0831 - val_accuracy: 0.9825
Epoch 3/20
701/701 [==============================] - 90s 129ms/step - loss: 0.0822 - accuracy: 0.9889 - val_loss: 0.1170 - val_accuracy: 0.9841
Epoch 4/20
701/701 [==============================] - 90s 129ms/step - loss: 0.0927 - accuracy: 0.9916 - val_loss: 0.2911 - val_accuracy: 0.9820
Epoch 5/20
701/701 [==============================] - 90s 129ms/step - loss: 0.0677 - accuracy: 0.9914 - val_loss: 0.1689 - val_accuracy: 0.9666

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/20
701/701 [==============================] - 91s 131ms/step - loss: 0.0227 - accuracy: 0.9946 - val_loss: 0.1556 - val_accuracy: 0.9758
Epoch 7/20
701/701 [==============================] - 90s 129

In the end, we can get accuracy about 99.5% on train dataset and 98.5 - 99% on the validation.